<a href="https://colab.research.google.com/github/imen-ch-20/PROJETASL/blob/master/ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

# =========================
# 1️⃣ Load and prepare data
# =========================
from google.colab import files
uploaded = files.upload()
def load_data(file_path):
    df = pd.read_excel(file_path)

    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()

    # Try to detect possible column name variants
    possible_start = [c for c in df.columns if "start" in c]
    possible_end = [c for c in df.columns if "end" in c]

    if not possible_start or not possible_end:
        raise ValueError("❌ Impossible de trouver les colonnes 'Start time' et 'End time' dans le fichier Excel.")

    start_col, end_col = possible_start[0], possible_end[0]

    # Convert time columns safely
    df[start_col] = pd.to_datetime(df[start_col].astype(str).str.strip(), format="%H:%M", errors="coerce")
    df[end_col] = pd.to_datetime(df[end_col].astype(str).str.strip(), format="%H:%M", errors="coerce")

    # Remove rows where time conversion failed
    df = df.dropna(subset=[start_col, end_col])

    # Convert to numeric hours (e.g. 6:30 → 6.5)
    df["start_hour"] = df[start_col].dt.hour + df[start_col].dt.minute / 60
    df["end_hour"] = df[end_col].dt.hour + df[end_col].dt.minute / 60

    # Create a random rating column (1–10) if it doesn't exist
    if "rating" not in df.columns:
        df["rating"] = np.random.randint(1, 10, len(df))

    # Ensure we have a 'day' column — if not, assign randomly
    if "day" not in df.columns:
        df["day"] = np.random.choice(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"], len(df))

    return df, start_col, end_col


# =========================
# 2️⃣ Fitness Function
# =========================
def fitness(schedule):
    overlap_penalty = 0
    for i in range(len(schedule)):
        for j in range(i + 1, len(schedule)):
            if (
                schedule[i]["day"] == schedule[j]["day"]
                and not (
                    schedule[i]["end_hour"] <= schedule[j]["start_hour"]
                    or schedule[j]["end_hour"] <= schedule[i]["start_hour"]
                )
            ):
                overlap_penalty += 1

    avg_rating = np.mean([c["rating"] for c in schedule])
    return avg_rating - 5 * overlap_penalty  # heavy penalty for overlaps


# =========================
# 3️⃣ Generate Neighbor
# =========================
def generate_neighbor(schedule):
    neighbor = schedule.copy()
    if len(schedule) > 1:
        i, j = random.sample(range(len(schedule)), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor


# =========================
# 4️⃣ Ant Colony Optimization
# =========================
def ant_colony_optimization(df, n_ants=20, n_iterations=60, alpha=1, beta=3, rho=0.1):
    n_courses = len(df)
    pheromone = np.ones((n_courses, n_courses))
    best_fitness = -np.inf
    best_schedule = None

    for iteration in range(n_iterations):
        all_schedules = []
        all_fitness = []

        for ant in range(n_ants):
            schedule = df.sample(frac=1).to_dict("records")
            fit = fitness(schedule)
            all_schedules.append(schedule)
            all_fitness.append(fit)

        iteration_best_idx = np.argmax(all_fitness)
        iteration_best_fit = all_fitness[iteration_best_idx]

        if iteration_best_fit > best_fitness:
            best_fitness = iteration_best_fit
            best_schedule = all_schedules[iteration_best_idx]

        pheromone = (1 - rho) * pheromone + rho * np.random.rand(n_courses, n_courses)
        print(f"Iteration {iteration + 1}/{n_iterations} — Best Fitness: {iteration_best_fit:.2f}")

    return best_schedule, best_fitness


# =========================
# 5️⃣ Run Optimization
# =========================
file_path = "/content/Timetabling-Optimisation-Solution.xlsx"
df, start_col, end_col = load_data(file_path)

best_schedule, best_fitness = ant_colony_optimization(df)

# =========================
# 6️⃣ Show Result as Table
# =========================
if best_schedule:
    result_df = pd.DataFrame(best_schedule)[
        ["module code", "module title", "day", start_col, end_col, "rating"]
    ]
    print("\n=== 📅 Meilleur emploi du temps trouvé ===")
    display(result_df.head(15))  # affiche les 15 premières lignes
    print(f"\n⭐ Score de fitness final : {best_fitness:.2f}")
else:
    print("❌ Aucun emploi du temps valide trouvé.")

